In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
import yfinance as yf
import pandas as pd
import numpy as pd
# import pyspark.sql.SQLContext
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql import Window

spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [3]:
tsla = yf.Ticker("TSLA")
tsla

yfinance.Ticker object <TSLA>

In [4]:
# apple = spark.createDataFrame(aapl)
tsla.info

{'zip': '94304',
 'sector': 'Consumer Cyclical',
 'fullTimeEmployees': 48016,
 'longBusinessSummary': 'Tesla, Inc. designs, develops, manufactures, leases, and sells electric vehicles, and energy generation and storage systems in the United States, China, Netherlands, Norway, and internationally. The company operates in two segments, Automotive; and Energy Generation and Storage. The Automotive segment offers sedans and sport utility vehicles. It also provides electric powertrain components and systems; and services for electric vehicles through its company-owned service locations, and Tesla mobile service technicians, as well as sells used vehicles. This segment markets and sells its products through a network of company-owned stores and galleries, as well as through its own Website. The Energy Generation and Storage segment offers energy storage products, such as rechargeable lithium-ion battery systems for use in homes, industrial, commercial facilities, and utility grids; and desig

In [5]:
hist = tsla.history(period="14d")
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-04-22,703.98,734.00,688.71,732.11,14224800,0,0
2020-04-23,727.60,734.00,703.13,705.63,13236700,0,0
2020-04-24,710.81,730.73,698.18,725.15,13237600,0,0
2020-04-27,737.61,799.49,735.00,798.75,20681400,0,0
2020-04-28,795.64,805.00,756.69,769.12,15222000,0,0
2020-04-29,790.17,803.20,783.16,800.51,16216000,0,0
2020-04-30,855.19,869.82,763.50,781.88,28471900,0,0
2020-05-01,755.00,772.77,683.04,701.32,32531800,0,0
2020-05-04,701.00,762.00,698.00,761.19,19237100,0,0


In [6]:
tesla = spark.createDataFrame(hist)

/Users/rich/dev/spark/spark-2.4.5-bin-hadoop2.7/python/pyspark/sql/session.py:714: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 0.8.0 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


In [7]:
tesla.describe()

DataFrame[summary: string, Open: string, High: string, Low: string, Close: string, Volume: string, Dividends: string, Stock Splits: string]

trying to show the ability to use spark sql below

In [11]:
sparkdf = sqlContext.createDataFrame(hist.reset_index(drop=False))
sparkdf

DataFrame[Date: timestamp, Open: double, High: double, Low: double, Close: double, Volume: bigint, Dividends: bigint, Stock Splits: bigint]

cell below is for potential NLP looping through a dframe.  To be continued

In [12]:
closings = []
for row in sparkdf.itertuples():
    for closing in row[2].split('.'):
        if closing != '':
            closing.append((row[1], closing))
new_df = pandas.sparkdf(closings, columns=['DATE', 'CLOSE'])

AttributeError: 'DataFrame' object has no attribute 'itertuples'

In [13]:
sparkdf.show()

+-------------------+------+------+------+------+--------+---------+------------+
|               Date|  Open|  High|   Low| Close|  Volume|Dividends|Stock Splits|
+-------------------+------+------+------+------+--------+---------+------------+
|2020-04-22 00:00:00|703.98| 734.0|688.71|732.11|14224800|        0|           0|
|2020-04-23 00:00:00| 727.6| 734.0|703.13|705.63|13236700|        0|           0|
|2020-04-24 00:00:00|710.81|730.73|698.18|725.15|13237600|        0|           0|
|2020-04-27 00:00:00|737.61|799.49| 735.0|798.75|20681400|        0|           0|
|2020-04-28 00:00:00|795.64| 805.0|756.69|769.12|15222000|        0|           0|
|2020-04-29 00:00:00|790.17| 803.2|783.16|800.51|16216000|        0|           0|
|2020-04-30 00:00:00|855.19|869.82| 763.5|781.88|28471900|        0|           0|
|2020-05-01 00:00:00| 755.0|772.77|683.04|701.32|32531800|        0|           0|
|2020-05-04 00:00:00| 701.0| 762.0| 698.0|761.19|19237100|        0|           0|
|2020-05-05 00:0

In [14]:
sparkdf.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: long (nullable = true)
 |-- Dividends: long (nullable = true)
 |-- Stock Splits: long (nullable = true)



In [15]:
sparkdf.count()

14

In [16]:
sparkdf.limit(5).show()

+-------------------+------+------+------+------+--------+---------+------------+
|               Date|  Open|  High|   Low| Close|  Volume|Dividends|Stock Splits|
+-------------------+------+------+------+------+--------+---------+------------+
|2020-04-22 00:00:00|703.98| 734.0|688.71|732.11|14224800|        0|           0|
|2020-04-23 00:00:00| 727.6| 734.0|703.13|705.63|13236700|        0|           0|
|2020-04-24 00:00:00|710.81|730.73|698.18|725.15|13237600|        0|           0|
|2020-04-27 00:00:00|737.61|799.49| 735.0|798.75|20681400|        0|           0|
|2020-04-28 00:00:00|795.64| 805.0|756.69|769.12|15222000|        0|           0|
+-------------------+------+------+------+------+--------+---------+------------+



In [17]:

sparkdf.createOrReplaceTempView("hist")


In [24]:
window = Window.partitionBy().orderBy(("Date").desc)
sparkdf = sparkdf.withColumn("Close_prev", F.lag(df.value).over(my_window))


AttributeError: 'str' object has no attribute 'desc'

In [20]:
show = """SHOW COLUMNS FROM hist"""
spark.sql(show).show()

+------------+
|    col_name|
+------------+
|        Date|
|        Open|
|        High|
|         Low|
|       Close|
|      Volume|
|   Dividends|
|Stock Splits|
+------------+



In [43]:
query = """SELECT *, 
LEAD(Close, 1) OVER (ORDER BY Close) as Close_prev 
FROM hist
"""
# sparkdf = sparkdf.withColumn("Close_prev", F.lag(sparkdf.Close))
# sparkdf = sparkdf.withColumn("Close_diff", F.when(F.isnull(sparkdf.Close - sparkdf.Close_prev), 0)
#                    .otherwise(sparkdf.Close - sparkdf.Close_prev))

drop_list = ['Dividends', 'Stock Splits']
query.select([column for column in query.columns if column not in drop_list])
spark.sql(query).show()
# import org.apache.spark.sql.expressions._
# windowSpec = Window.partitionBy("Column1").orderBy("Date_Converted")
# import org.apache.spark.sql.functions._
# df.withColumn("diff_Amt_With_Prev_Month", $"Amount" - when((lag("Amount", 1).over(windowSpec)).isNull, 0).otherwise(lag("Amount", 1).over(windowSpec)))
#    .show(false)

AttributeError: 'str' object has no attribute 'select'

In [42]:
drop_list = ['Dividends', 'Stock Splits']
query.select([column for column in query.columns if column not in drop_list])

AttributeError: 'str' object has no attribute 'select'

Define MYSQL conneciton properties

In [54]:
sparkdf.write.jbdc(
    url='jdbc:mysql://localhost:3306/stocks',
    table='mystock',
    mode='append',
    properties={'user':'root','password':'yourpassword','driver':'com.mysql.cj.jdbc.Driver'}

)

AttributeError: 'DataFrameWriter' object has no attribute 'jbdc'